In [ ]:
import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
import statsmodels.formula.api as sm

from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score
from sklearn.ensemble import RandomForestRegressor
from sklearn import preprocessing
from scipy import stats
import lightgbm as lgb
import gc

In [ ]:
train_init= pd.read_csv('../input/train_V2.csv')

In [ ]:
#removing useless features
list_largest_corr = train_init.drop(['matchId','groupId','teamKills',u'winPlacePerc'],axis=1).columns

In [ ]:
train_without_out = train_init[(np.abs(stats.zscore(train_init)) < 6).all(axis=1)]
del train_init
gc.collect()
def add_means(train_without_out,list_largest_corr,isSampleTest=False):
    #mean stats by groupId
    #based in song kernel
    agg = train_without_out.groupby(['matchId','groupId'])[list_largest_corr].agg('mean')
    agg_rank = agg.groupby('matchId')[list_largest_corr].rank(pct=True).reset_index()
    train_without_out = train_without_out.merge(agg_rank, suffixes=["", "_mean_rank"], how='left', on=['matchId', 'groupId'])
    del agg
    del agg_rank
    gc.collect()
    #size group
    agg = train_without_out.groupby(['matchId','groupId']).size().reset_index(name='group_size')
    train_without_out = train_without_out.merge(agg, how='left', on=['matchId', 'groupId'])
    del agg
    gc.collect()
    
    #size match
    agg = train_without_out.groupby(['matchId']).size().reset_index(name='match_size')
    train_without_out = train_without_out.merge(agg, how='left', on=['matchId'])
    del agg
    gc.collect()
    
    #mean stats by match
    agg = train_without_out.groupby(['matchId'])[list_largest_corr].agg('mean').reset_index()
    train_without_out = train_without_out.merge(agg,suffixes=["", "_mean_match"], how='left', on=['matchId'])
    del agg
    gc.collect()
    
    if isSampleTest == False:
        x = train_without_out.drop('winPlacePerc',axis=1)
    else: 
        x = train_without_out
    scaler = preprocessing.MinMaxScaler(feature_range=(0, 1), copy=False).fit(x)
    scaler.transform(x)
    return x
x = add_means(train_without_out,list_largest_corr)
y = train_without_out['winPlacePerc']
del train_without_out
gc.collect()

In [ ]:
# knn = RandomForestRegressor(n_jobs=-1)
# knn.fit(x,y)
# knn.set_params(n_estimators=60)

In [ ]:
d_train = lgb.Dataset(x, y)
iterations = 2000
watchlist = [d_train]
#parameters based on LightGBM Starter Notebook
params = {
    'learning_rate': 0.1, 
    'max_depth': -1,
    'num_leaves': 30,
    'feature_fraction': 0.9,
    'min_data_in_leaf': 100,
    'lambda_l2': 4,
    'objective': 'regression_l2', 
    'metric': 'mae',
    'seed': 123}

model = lgb.train(params, train_set=d_train, num_boost_round=iterations, 
        valid_sets=watchlist, early_stopping_rounds=20, verbose_eval=200) 

In [ ]:
test = pd.read_csv('../input/test.csv')
x_test = add_means(test,list_largest_corr,True)
predict = model.predict(x_test)

In [ ]:
test['winPlacePerc'] = predict
test[['Id','winPlacePerc']].to_csv("submission.csv", index = False)

In [ ]:
test[['Id','winPlacePerc']].head()